In [ ]:
# calculate FID similarity
import os
from pytorch_fid import fid_score

# set env
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
src,dst = [],[]
src.append('./image_log/val_last_deconv0/0_samples_cfg_scale_9.10')
src.append('./image_log/val_last_deconv1/1_samples_cfg_scale_9.10')
src.append('./image_log/val_last_deconv2/2_samples_cfg_scale_9.10')
src.append('./image_log/val_last_deconv3/3_samples_cfg_scale_9.10')
src.append('./image_log/val_last_deconv4/4_samples_cfg_scale_9.10')
src.append('./image_log/val_last_deconv5/5_samples_cfg_scale_9.10')
src.append('./image_log/val_last_canny/5_samples_cfg_scale_9.10')
dst.append('./image_log/val_last_deconv0/reconstruction')
dst.append('./image_log/val_last_deconv1/reconstruction')
dst.append('./image_log/val_last_deconv2/reconstruction')
dst.append('./image_log/val_last_deconv3/reconstruction')
dst.append('./image_log/val_last_deconv4/reconstruction')
dst.append('./image_log/val_last_deconv5/reconstruction')
dst.append('./image_log/val_last_canny/reconstruction')

for s,d in zip(src,dst):
    fid_value = fid_score.calculate_fid_given_paths([s,d],batch_size=1000,device='cuda',dims=2048)

    print('FID value:',fid_value)

In [ ]:
# calculate CLIP similarity

import torch
import clip
from PIL import Image
import torch.nn as nn
import os
from evaluate.DataProvider import data_provider
from einops import rearrange,reduce

# set env and model
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)
cos = torch.nn.CosineSimilarity(dim=-1)
model.eval()

# set image file
src,dst = [],[]
src.append('./image_log/val_last_deconv0/0_samples_cfg_scale_9.10')
src.append('./image_log/val_last_deconv1/1_samples_cfg_scale_9.10')
src.append('./image_log/val_last_deconv2/2_samples_cfg_scale_9.10')
src.append('./image_log/val_last_deconv3/3_samples_cfg_scale_9.10')
src.append('./image_log/val_last_deconv4/4_samples_cfg_scale_9.10')
src.append('./image_log/val_last_deconv5/5_samples_cfg_scale_9.10')
src.append('./image_log/val_last_canny/5_samples_cfg_scale_9.10')
dst.append('./image_log/val_last_deconv0/reconstruction')
dst.append('./image_log/val_last_deconv1/reconstruction')
dst.append('./image_log/val_last_deconv2/reconstruction')
dst.append('./image_log/val_last_deconv3/reconstruction')
dst.append('./image_log/val_last_deconv4/reconstruction')
dst.append('./image_log/val_last_deconv5/reconstruction')
dst.append('./image_log/val_last_canny/reconstruction')

# calculate mean CLIP similarity between two image sets
with torch.no_grad():
    for s,d in zip(src,dst):
        ds1,ds2 = data_provider(s,batch_size=1000,dstype='CLIP'),data_provider(d,batch_size=1000,dstype='CLIP')
        print(s+","+d)
        sim = []
        for batch1,batch2 in zip(ds1,ds2):
            b1,b2 = batch1.to(device),batch2.to(device)
            f1,f2 = model.encode_image(b1),model.encode_image(b2)
            similarity = (cos(f1,f2)+1.)/2
            sim.append(similarity)

        si = reduce(sim,'n b -> ()','mean').item()
        print('similarity: ',si)
        
        # break


In [1]:
# calculate SSIM similarity

from torchmetrics.image import StructuralSimilarityIndexMeasure as SSIM
from torchmetrics.image import StructuralSimilarityIndexMeasure as SSIM
from torchmetrics.image import LearnedPerceptualImagePatchSimilarity as LPIPS
from torchmetrics.image import PeakSignalNoiseRatio as PSNR
import torch
import os
from evaluate.DataProvider import data_provider
from einops import rearrange,reduce


# set env and model
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
device = "cuda" if torch.cuda.is_available() else "cpu"
ssim = SSIM(kernel_size=11,k1=0.01,k2=0.03).to(device)
lpips = LPIPS(net_type='vgg').to(device)
psnr = PSNR().to(device)


# set image file
src,dst = [],[]
src.append('./image_log/val_last_deconv0/0_samples_cfg_scale_9.10')
src.append('./image_log/val_last_deconv1/1_samples_cfg_scale_9.10')
src.append('./image_log/val_last_deconv2/2_samples_cfg_scale_9.10')
src.append('./image_log/val_last_deconv3/3_samples_cfg_scale_9.10')
src.append('./image_log/val_last_deconv4/4_samples_cfg_scale_9.10')
src.append('./image_log/val_last_deconv5/5_samples_cfg_scale_9.10')
src.append('./image_log/val_last_canny/5_samples_cfg_scale_9.10')
dst.append('./image_log/val_last_deconv0/reconstruction')
dst.append('./image_log/val_last_deconv1/reconstruction')
dst.append('./image_log/val_last_deconv2/reconstruction')
dst.append('./image_log/val_last_deconv3/reconstruction')
dst.append('./image_log/val_last_deconv4/reconstruction')
dst.append('./image_log/val_last_deconv5/reconstruction')
dst.append('./image_log/val_last_canny/reconstruction')


with torch.no_grad():
    for s,d in zip(src,dst):
        ds1,ds2 = data_provider(s,batch_size=50,dstype='SSIM',color='L'),data_provider(d,batch_size=50,dstype='SSIM',color='L')
        print(s+","+d)
        sim = []
        for batch1,batch2 in zip(ds1,ds2):
            b1,b2 = batch1.to(device),batch2.to(device)
            # print(lpips(b1,b2))
            sim.append(psnr(b1,b2))

        si = reduce(sim,'n -> ()','mean').item()
        print('similarity: ',si)


./image_log/val_last_deconv0/0_samples_cfg_scale_9.10,./image_log/val_last_deconv0/reconstruction
similarity:  19.160648345947266
./image_log/val_last_deconv1/1_samples_cfg_scale_9.10,./image_log/val_last_deconv1/reconstruction
similarity:  19.902210235595703
./image_log/val_last_deconv2/2_samples_cfg_scale_9.10,./image_log/val_last_deconv2/reconstruction
similarity:  17.936491012573242
./image_log/val_last_deconv3/3_samples_cfg_scale_9.10,./image_log/val_last_deconv3/reconstruction
similarity:  14.830964088439941
./image_log/val_last_deconv4/4_samples_cfg_scale_9.10,./image_log/val_last_deconv4/reconstruction
similarity:  11.520975112915039
./image_log/val_last_deconv5/5_samples_cfg_scale_9.10,./image_log/val_last_deconv5/reconstruction
similarity:  11.23917293548584
./image_log/val_last_canny/5_samples_cfg_scale_9.10,./image_log/val_last_canny/reconstruction
similarity:  10.695695877075195
